In [45]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join

In [46]:
book_folder = "C:/Praveen/deribit/book_snapshot_5/"
trade_folder = "C:/Praveen/deribit/trades/"

In [47]:
modified_book_folder = "C:/Praveen/Python/AlphaHFT/temp_books/"
modified_trade_folder = "C:/Praveen/Python/AlphaHFT/temp_trades/"

In [48]:
data_folder = "C:/Praveen/Python/AlphaHFT/"

In [49]:
books = [join(book_folder, f) for f in listdir(book_folder) if isfile(join(book_folder, f))]
for book in books:
    tokens = os.path.basename(book).split("_")
    date_holder = tokens[4]
    symbol = tokens[5]
    trade = r"deribit_trades_{0}_{1}".format(date_holder, symbol)
    trade = join(trade_folder, trade)
    book_df = pd.read_csv(book, header=0, index_col='local_timestamp')
    trade_df = pd.read_csv(trade, header=0, index_col='local_timestamp')
    data = join(data_folder, r"data_{0}_{1}".format(date_holder, symbol))
    data_df = pd.read_csv(data, header=0, index_col='timestamp')
    break

In [50]:
#verify books
def validate_prices(i, curr_ts, data_df, book):
    ds = data_df.loc[curr_ts, :]
    
    if ds.loc["bid_price"] != book.loc["bids[0].price"]:
        print("bid 0 mismatch: ", i)
        print(ds.loc["bid_price"], book.loc["bids[0].price"], bdx[-1])
        
    if ds.loc["ask_price"] != book.loc["asks[0].price"]:
        print("ask 0 mismatch: ", i)
        print(ds.loc["ask_price"], book.loc["asks[0].price"], bdx[-1])

    bid_1 = ds.loc["bid_price"] * (1 + ds.loc["bid_ticks_1"] / 1000)
    if not np.allclose(bid_1, book.loc["bids[1].price"]):
        print("bid 1 mismatch: ", i)

    ask_1 = ds.loc["ask_price"] * (1 + ds.loc["ask_ticks_1"] / 1000)
    if not np.allclose(ask_1, book.loc["asks[1].price"]):
        print("ask 1 mismatch: ", i)

    bid_2 = ds.loc["bid_price"] * ds.loc["bid_ticks_2"] / 1000 + bid_1
    if not np.allclose(bid_2, book.loc["bids[2].price"]):
        print("bid 2 mismatch: ", i)

    ask_2 = ds.loc["ask_price"] * ds.loc["ask_ticks_2"] / 1000 + ask_1
    if not np.allclose(ask_2, book.loc["asks[2].price"]):
        print("ask 2 mismatch: ", i)

    bid_3 = ds.loc["bid_price"] * ds.loc["bid_ticks_3"] / 1000 + bid_2
    if not np.allclose(bid_3, book.loc["bids[3].price"]):
        print("bid 3 mismatch: ", i)

    ask_3 = ds.loc["ask_price"] * ds.loc["ask_ticks_3"] / 1000 + ask_2
    if not np.allclose(ask_3, book.loc["asks[3].price"]):
        print("ask 3 mismatch: ", i)

    bid_4 = ds.loc["bid_price"] * ds.loc["bid_ticks_4"] / 1000 + bid_3
    if not np.allclose(bid_4, book.loc["bids[4].price"]):
        print("bid 4 mismatch: ", i)

    ask_4 = ds.loc["ask_price"] * ds.loc["ask_ticks_4"] / 1000 + ask_3
    if not np.allclose(ask_4, book.loc["asks[4].price"]):
        print("ask 4 mismatch: ", i)

ts = None
book_df.index = pd.to_datetime(book_df.index, unit='us')
book_df = book_df.resample("1s").last().ffill()

for i in range(0, data_df.shape[0]):
    curr_ts = data_df.index[i]
    bdx = book_df.index[book_df.index == curr_ts]

    book = book_df.loc[bdx[-1], :]
    validate_prices(i, curr_ts, data_df, book)

In [51]:
data_df.columns

Index(['bid_incr', 'ask_incr', 'bid_ticks_1', 'bid_ticks_2', 'bid_ticks_3',
       'bid_ticks_4', 'ask_ticks_1', 'ask_ticks_2', 'ask_ticks_3',
       'ask_ticks_4', 'bid_sum_0', 'bid_sum_1', 'bid_sum_2', 'bid_sum_3',
       'bid_sum_4', 'ask_sum_0', 'ask_sum_1', 'ask_sum_2', 'ask_sum_3',
       'ask_sum_4', 'bid_sum_diff_0', 'bid_sum_diff_1', 'bid_sum_diff_2',
       'bid_sum_diff_3', 'bid_sum_diff_4', 'ask_sum_diff_0', 'ask_sum_diff_1',
       'ask_sum_diff_2', 'ask_sum_diff_3', 'ask_sum_diff_4', 'bid_price',
       'ask_price', 'long_size_diff', 'short_size_diff', 'long_price_diff',
       'short_price_diff'],
      dtype='object')

In [52]:
data_df.shape

(86392, 36)

In [53]:
data_df.head(2)

,bid_incr,ask_incr,bid_ticks_1,bid_ticks_2,bid_ticks_3,bid_ticks_4,ask_ticks_1,ask_ticks_2,ask_ticks_3,ask_ticks_4,...,ask_sum_diff_1,ask_sum_diff_2,ask_sum_diff_3,ask_sum_diff_4,bid_price,ask_price,long_size_diff,short_size_diff,long_price_diff,short_price_diff
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:08,0.0,0.0,-0.070947,-0.047298,-0.047298,-0.011825,0.023649,0.023649,0.011824,0.035473,...,-0.035152,-0.031377,-0.027452,-0.018593,42285.0,42285.5,0.0,-0.906783,0.0,-0.095891
2024-01-01 00:00:09,0.0,0.0,-0.070947,-0.047298,-0.047298,-0.011825,0.011824,0.011824,0.023649,0.011824,...,0.093249,0.044053,0.023520,0.024843,42285.0,42285.5,0.0,-0.906783,0.0,-0.095891


In [54]:
data_df.tail(2)

,bid_incr,ask_incr,bid_ticks_1,bid_ticks_2,bid_ticks_3,bid_ticks_4,ask_ticks_1,ask_ticks_2,ask_ticks_3,ask_ticks_4,...,ask_sum_diff_1,ask_sum_diff_2,ask_sum_diff_3,ask_sum_diff_4,bid_price,ask_price,long_size_diff,short_size_diff,long_price_diff,short_price_diff
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-01 23:59:58,0.0,0.0,-0.022612,-0.011306,-0.022612,-0.022612,0.169589,0.011306,0.022612,0.022612,...,0.054687,0.033500,0.037610,0.035975,44224.0,44224.5,0.627342,0.0,0.0,0.0
2024-01-01 23:59:59,0.0,0.0,-0.022612,-0.033918,-0.033918,-0.011306,0.146977,0.022612,0.033918,0.022612,...,-0.902317,-0.825368,-0.797121,-0.738954,44224.0,44224.5,-0.998851,0.0,0.0,0.0
